In [ ]:
!pip install flair

In [34]:
!pip install tqdm


# Zero Shot Learning (ZSL)

In [201]:
import pandas as pd
import flair
import tqdm

from itertools import islice


In [214]:
df = pd.read_csv("to_ZSL.csv")

In [215]:
df.head()

,book_authors,book_title,book_desc,genre
0,Suzanne Collins,The Hunger Games,Winning will make you famous. Losing means cer...,Young Adult
1,J.K. Rowling|Mary GrandPré,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,Fantasy
2,Harper Lee,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,Classics
3,Stephenie Meyer,Twilight,About three things I was absolutely positive.F...,Young Adult
4,Markus Zusak,The Book Thief,Trying to make sense of the horrors of World W...,Historical


In [216]:
from flair.models import TARSClassifier
from flair.data import Sentence

In [217]:
# TARS: Task-aware representation of sentences
tars = TARSClassifier.load('tars-base')


2022-05-24 09:55:36,680 loading file /root/.flair/models/tars-base-v8.pt


In [218]:
# the labels to be applied
# try another - smaller - taxonomy
genres = df.genre.unique().tolist()

In [221]:
df_pred_genres = df.copy()


In [222]:
def genre_tagger(summary, genre, tars):
  s = Sentence(summary)
  tars.predict_zero_shot(s, genre, multi_label=True)
  dict_tag = {}
  for label in s.labels:
      dict_tag[label.value] = label.score
  dict_tag = {k: dict_tag[k] for k in sorted(dict_tag, key=dict_tag.get, reverse=True)}
  top_genres = {k: dict_tag[k] for k in list(dict_tag)[:3]}


  return top_genres
  

In [ ]:
df_pred_genres[['tag_1', 'tag_2', 'tag_3']] = df_pred_genres["book_desc"].apply(lambda x: genre_tagger(x, genres, tars))

In [ ]:
df_pred_genres

# Few Shot Learning (FSL)

# NER x ZSL

In [ ]:
def ner_extraction(summary, model): 
  s = Sentence(summary)
  tars.predict(s)
  print(sentence.to_tagged_string("ner"))